In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import SAVE_FOLDER

## Load region-product export data

In [ ]:
trade = pd.read_parquet(f'{SAVE_FOLDER}/cleaned.parquet')
trade.head()

In [ ]:
## function that take the parameter, do linear to log transform and return dataset
def transform(R0,trade):
    df = trade.assign(llrca = np.log(1+trade.rca/R0)/np.log(1+1/R0))
    df['avgrca_part_p'] = df.llrca*df.pivec_p
    df['avgrca_p'] = df.groupby(['year','region'])['avgrca_part_p'].transform(sum)
    df['rct_p'] = np.where(df.avgrca_p>0,df.llrca/df.avgrca_p,0)
    df['rct_demean_p'] = df.rct_p-df.groupby('prod').rct_p.transform('mean')
    df['projpart_demean'] = df.pcil_p*df.rct_demean_p
    df['proj_demean'] = df.groupby(['year','region'])['projpart_demean'].transform(sum)
    df['loading_demean'] = df.proj_demean*df.pci_p
    return (np.square(df.loading_demean).sum(),np.square(df.loading_demean).sum()/np.square(df.rct_demean_p).sum())

In [ ]:
## loop over the possible range of parameter
res = np.zeros((40,2))
R0s = 10**np.arange(-8,2,0.25)
for i in range(40):
    res[i,:] = transform(R0s[i],trade)

In [ ]:
## plot the result
plt.plot(np.log10(R0s),res[:,1])
plt.xlabel("log10(R0)")
plt.ylabel("Var(ECI*) ratio")

R0 = 0.115 at the peak was chosen as parameter for future analysis